# Multiple Linear Regression of Cryptocurrency

The following model I have developed to demonstrate how to extract, transform and load data into a pipeline to solve a Multiple Regression problem. I aim to forecast Bitcoin (C$) prices by extrapolating historical data, training, testing and validating the model before evaluating its accuracy using a benchmark score.

I am looking to predict the closing price (dependent variable) based on its relationship with other intraday price and volume information (independent variables).

## Performance Measure Selection
For the purpose of a multiple regression model I am selecting the Root Mean Square Error (RMSE) to measure the overall degree of error when making my predictions. This will act as a measure of the 'Loss', or 'Cost' function and will help assess the degree of variability associated between my target and predictor variables.

$$ RMSE(X,h) = \sqrt {\frac{1}{m} \sum_{i=1}^{m}(h(x^ {i})-y^ {i})^2} $$

In [ ]:
import pandas as pd
import numpy as np
import os
import tarfile
import urllib3

DOWNLOAD_URL = "https://ca.finance.yahoo.com/quote/BTC-CAD/history?p=BTC-CAD"
BITCOIN_PATH = os.path.join("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")
BITCOIN_URL = DOWNLOAD_URL + "C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv"

The uniform resource locater for the downloaded dataset can be changed according to any Ticker symbol listed in Yahoo Finance. The path to extract the data to can be stored in a separate variable which can then be joined to the download URL and stored in yet another container. This just makes it a little easier to reference an object for compression-decompression 'tarfiles' which may save processing time and reduce latency.

In [ ]:
def fetch_bitcoin_data(bitcoin_url=BITCOIN_URL, bitcoin_path=BITCOIN_PATH):
    os.makedirs(bitcoin_path, exist_ok=True)
    tgz_path = os.path.join(bitcoin_path)
    urllib3.request.urlretrieve(bitcoin_url, tgz_path)
    bitcoin_tgz = tarfile.open(tgz_path)
    bitcoin_tgz.extract_all(path=bitcoin_path)
    bitcoin_tgz.close()

def load_bitcoin_data(bitcoin_path=BITCOIN_PATH):
    csv_path = os.path.join(bitcoin_path)
    return pd.read_csv(csv_path)

This works for large files which need compressing but an easier alternative for smaller file sizes (maybe less than 10,000 entries) would be to extract the dataset's URL, parse the data in CSV format and then store it as a dataframe variable using Pandas:

In [ ]:
'''
# import data
bitcoin_path = "https://ca.finance.yahoo.com/quote/BTC-CAD/history?p=BTC-CAD"

# read data into csv chosen format
bitcoin = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")

#load data into dataframe
df = pd.DataFrame(bitcoin).dropna(axis=0)
'''

Start by loading the Canadian denominated Bitcoin data below.

In [ ]:
btc_cad = load_bitcoin_data()

Each column represents a different feature all of which are listed below. These features are also known as 'attributes', 'input', or 'predictor' variables which can be used to ascertain a stock price prediction (called 'label', 'output', or 'target' variables). Just to be clear, once a 'target' variable is chosen for output, we now have a whole columnal vector containing elements or values for that particular feature which can be used in the linear regression model. These values are used as 'labeled' data.

In [ ]:
# get stock info
print(btc_cad.info())

Just to explain briefly, all the columns in this table contain float variables except for the 'Date' column which contains string objects.

## Data Preprocessing
### Overall Statistics
Using the describe() method to view summary statistics helps define the average values or range of data involved in this dataset, as well as a measure of dispersion about its mean.

In [ ]:
btc_cad.describe()

So I can tell the average closing price is C$29,865 to the nearest integer. 

There isn't much data to work from (one years worth exactly), but I would like to create a dataframe object from the price data and try and use it to forecast future price information. Taking a look at which columns are present in the dataset gives me:

In [ ]:
btc_cad.columns

And now lets see the first and last 5 entries in the table:

In [ ]:
btc_cad.head()

And now the tail of the dataframe.

In [ ]:
btc_cad.tail()

## Cleaning the Data
A brief look at the summary info for the dataset tells me there are a different number of date values compared to the rest of the column entries. This will need to be addressed, followed by any missing values, NaN, 'isnull' or entries with 'zero'. I can also see the different number 'count' for entries using the count() method:

In [ ]:
btc_cad.count()

First I just want to see what shape the dataframe is.

In [ ]:
btc_cad.shape

So it's 366 rows by 7 columns or a (366x7) matrix. Next, to find the total number of missing entries.

In [ ]:
bitcoin_missing_values_count = btc_cad.isnull().sum()
bitcoin_missing_values_count[0:7]

Further examination allows me to calculate the total number of rows containing missing data points as a percentage of the total number of rows in the dataframe.

In [ ]:
total_bitcoin = np.product(btc_cad.shape)
total_missing_bitcoin = bitcoin_missing_values_count.sum()

percent_missing_values = (total_missing_bitcoin/total_bitcoin) * 100
print(percent_missing_values)

From this I can see that just under 1% of the total values are missing. I can either choose to leave them out completely (as they may not really affect my model for the purpose of this exercise) or use imputed values such as the mean or median.

### Removing Rows
Having checked the csv file I can determine that there is no real reason for the missing values (null entries) for these dates. At first I thought these missing values might be due to public holidays but because Bitcoin is a currency, it appears to be traded OTC (Over The Counter) beyong normal trading hours throughout the day, seven days a week. As a result I have chosen to leave these missing values out using the dropna() method to eliminate them. This should provide a total of 162 rows.

In [ ]:
# dropping the rows with missing or 'na' entries using 'axis=0' for the row dimension
btc_cad = btc_cad.dropna(axis=0)

In [ ]:
# finding the new shape of the data frame
btc_cad.shape

This time there are exactly 362 rows for each column. It's important to set the argument (axis=0) otherwise dropping the values doesn't work. 

### Removing Columns (Feature Selection)

Having analyzed the CSV file I can see the values in the 'Adj Close' column are exactly the same as in the 'Close' column, so I will remove 'Adj Close' completely from the dataset in an attempt to regularize the model. This is unecessary data which can be removed as it could contribute to bias and bleed into the test data. It's better to reduce the overall number of variables particularly removing those which are not relevant (the process of regularization to simplify the model and avoid overfitting).

Taking a look at all the columns in the dataframe:

In [ ]:
btc_cad.columns

In [ ]:
# delete the unwanted columns
del btc_cad['Adj Close']

Another way to achieve this would be to use the drop method().

In [ ]:
# check the new dataframe features
btc_cad.columns

In [ ]:
print(type(btc_cad))

# Exploratory Data Analysis

The mean value for the Bitcoin (CAD) close price is C$ 29,864.75 and from the summary statistics using 'describe()' I can also see the maximum values in the upper-quartile range have been tested regularly.

As there are no dividends or stock splits because Bitcoin is a currency (medium of exchange / store of value), only 6 columns are of any use to me in this particular dataset: Date, Open, High, Low, Close and Volume. Printing the first 5 lines of the dataframe gives:

In [ ]:
print(btc_cad[:5])

Next, I will store the independent column variables (features) in a newly named Pandas dataframe called 'bitcoin' which can be used for further analysis and is something which is just easier to remember.

In [ ]:
# re-naming the btc_cad dataframe as a new variable 'bitcoin'
bitcoin = btc_cad

In [ ]:
print(btc_cad)

In [ ]:
print(bitcoin)

## Correlation (Multivariate Stats)
The first stage of any feature selection process for a linear regression model should involve checking multivariate statistics and the degree of correlation between the features and also with the target variable. This multivariate statistic is essential to avoid poor performance as high correlations between variables can prevent the model's loss function from converging to the global minimum.

In [ ]:
correlation = bitcoin.corr()
print(correlation)

I would expect the opening and closing prices to be defined by the range of the high and low daily prices so all of these values tend to move within a fairly narrow band. There is a strong degree of positive correlation between these time-series values. I can also determine there is a less powerful degree of association between the volume of currency traded and its closing price (0.582709), but a positive relationship nonetheless. 

 Assuming I wanted to find the top three highest correlated features I could use the .abs() and .nlargest() methods:

In [ ]:
print(bitcoin.corr().abs().nlargest(3, 'Close').index)          # to find the top 3 features with highest correlation values

In [ ]:
print(bitcoin.corr().abs().nlargest(3, 'Close').values[:,3])         # top 3 correlation values

Firstly, the perfectly correclated value of 1.0 represents the 'Close' price correlation with itself, but the nest two correlation values of 0.99918803 and 0.99873489 represent those of the daily 'High' and 'Low' resepectively.

Remember, this would only be necessary if I wanted to include only those features with the top 3 correlation scores. For the purpose of this model that would be the open, high and low prices but I want to include the volume also and have decided to keep all 4 features. Next I intend to look at a scatter plot to show the relationship between some of these variables starting with the daily high and close prices.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.scatter(bitcoin['High'], bitcoin['Close'], marker='o')
plt.xlabel('High')
plt.ylabel('Close')
plt.show()

plt.scatter(bitcoin['Low'], bitcoin['Close'], marker='o', color='green')
plt.xlabel('Low')
plt.ylabel('Close')
plt.show()

plt.scatter(bitcoin['Open'], bitcoin['Close'], marker='o', color='orange')
plt.xlabel('Open')
plt.ylabel('Close')
plt.show()

A strong positive linear relationship exists between the High, Low and Open prices with the Close but the degree of correlation decreases with High, Low and Open prices respectively. Also this can be checked against the 4th row of the correlation matrix under the row with 'Close' in the entry.

And finally the relationship plot of daily volume against the closing prices. I can see there is one outlier on the right hand side.

In [ ]:
plt.scatter(bitcoin['Volume'], bitcoin['Close'], marker='o', color='coral')
plt.xlabel('Volume')
plt.ylabel('Close')
plt.show()

So there definitely is a relationship between 'Volume' and 'Close' but it doesn't appear to be very linear. It's all confined to a group with an outlier around the C$60,000 mark which represents a spike in contracts traded. Another problem I can see on the plot is the scale of the axis containing the 'Volume' values which indicates the need for feature normalization to address any adverse effects these values might have on the model training process.

## Feature Normalization
I first need to define the values of the numeric columns in the dataset to a common scale using MinMaxScaler(). This will make data exploration visualizations easier to interpret.

In [ ]:
import pandas as pd
from sklearn import preprocessing

This transformation will shift the values to a range between 0 and 1 for each column. 

In [ ]:
# remove row entries which aren't available
bitcoin = bitcoin.dropna(axis=0)

# assign feature columns
bitcoin.columns = ['Date','Open','High','Low','Close','Volume']

# convert data type from string to date format
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])
bitcoin.set_index('Date', inplace=True)

print(bitcoin['Date'])
'''
# convert entries to float type
X = bitcoin.values.astype(float)

# define min max scaler
min_max_scaler = preprocessing.MinMaxScaler()

# transform data
X_scaled = min_max_scaler.fit_transform(X)

bitcoin = pd.DataFrame(X_scaled, columns=bitcoin.columns)
print(bitcoin)
'''


## Visualizing the Data
Let's take a look at plotting the different price data over time. This first plot will have the different OHLC prices for the last year. For the purpose of displaying this particular line plot I will remove the 'Adj Close' because it's not needed and the 'Volume' feature as the scale is completely different to that of the OHLC prices.

In [ ]:
# Import the seaborn library
import seaborn as sns

# Location of the file path
#bitcoin_path = os.path.join("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")

# Open the file into a variable bitcoin
#bitcoin = pd.read_csv(bitcoin_path, index_col="Date", parse_dates=True)

# Plot the width and height of the figure
plt.figure(figsize=(14,6))

# Add a decsriptive title
plt.title("Daily Bitcoin OHLC Prices in $CAD")

# Line chart showing daily global streams of each song 
sns.lineplot(data=bitcoin.columns)

At first glance I can detect an underlying trend for the price of Bitcoin, but there is also a noticable spike in Feb 2021 before reverting back to the trend. This makes the price data easier to read once the width and height of the axes have been adjusted. If I want to create a subplot of the 'Close' price (as the target variable I will eventually use), then I may decide to list the columns first:

In [ ]:
list(bitcoin.columns)

In [ ]:
# Once again setting the width and height of the figure
plt.figure(figsize=(14,6))

# Add title
plt.title("Daily Bitcoin Close Price in $CAD")

# Line chart showing daily 'Close'
sns.lineplot(data=bitcoin['Close'], label="Close")

# Add label for horizontal axis
plt.xlabel("Date")

So as we can see there is a definite gradual upward trend. In this particular instance the plot exhibits a price pattern called an 'ascending triangle' which is a term used by technical analysts. This term can be used to make buying and selling decisions based on the shape of the pattern.

## Price Frequency Distributions and Data Dispersion

Taking a look at the frequency distributions of the different prices can tell me about their distributions about the average.

I know that earlier in the timeframe captured there were a large number of cumulative values within the ten to twenty thousand dollar range. The extra weighting given to these observations have obviously caused a fairly low mean value of approximately thirty thousand dollars, but the cryptocurrency finished up at 71.5 thousand dollars by April 2021. The overall range of price values has been divided into 20 separate bins for some clarity, so I can view the frequency or number of daily prices that fall within each interval category price range or bin.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style
import seaborn as sns

# create style
style.use("seaborn-darkgrid")

# set width and height
plt.figure(figsize=(10,6))

# Add a title
plt.title("Histogram of 'Close' Prices")

# Show the average daily closing price
sns.histplot(data=bitcoin, x=bitcoin['Close'], bins=20, color='darkturquoise')

# Add labels for both axes
plt.xlabel('Close Price')
plt.ylabel('Count')

Storing these values in a dataframe series and applying the mean method requires a few steps.

In [ ]:
# create filepath for dataset
btc_cad_path = os.path.join("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")

# create a dict and then store a single column as a dataframe
data = {'Open': bitcoin['Open'],
        'High': bitcoin['High'],
        'Low': bitcoin['Low'],
        'Close': bitcoin['Close']}

df = pd.DataFrame(data, columns = ['Close'])

print(df)
print (type(df))

Next, converting the DataFrame column to a Series using the squeeze() method:

In [ ]:
close_series = df['Close'].squeeze()

print(close_series)
print (type(close_series))

So, now the 'dataframe' column has been converted to a 'series' data type, I aim to find the mean of the values for 'Close' prices over the period in question and display their frequency distribution.

In [ ]:
# create style
style.use("seaborn-whitegrid")

# print the mean value
print(close_series.mean())

# configure the plot size
plt.figure(figsize=(10,6))

# create a title
plt.title("Frequency Distribution of Mean 'Close' Prices")

# plot the series
close_plot = sns.distplot(close_series, bins=20, color='red')

The mean value is to the right of the peak on the frequency curve with a long tail containing values toward C$90,000 which exhibits positive skewness. 

The frequency distribution plots for the open, high, low and daily volume not only explain the range of values and the dispersion about the mean, but they also be some indication of how prices might revert back to some long term average. It would be foolish to assume that Bitcoin prices will continue the current upward trend beyond C$80,000 because the average is so low.

In [ ]:
# create style
style.use("seaborn-whitegrid")

# set width and height
plt.figure(figsize=(10,6))

# Add a title
plt.title("Frequency Distribution of 'Open' Prices")

# Show the average daily high price
sns.histplot(data=bitcoin, x=bitcoin['Open'], bins=20, color='turquoise')

# Add labels for both axes
plt.xlabel('Open Price')
plt.ylabel('Count')

In [ ]:
# create style
style.use("seaborn-whitegrid")

# set width and height
plt.figure(figsize=(10,6))

# Add a title
plt.title("Frequency Distribution of 'High' Prices")

# Show the average daily high price
sns.histplot(data=bitcoin, x=bitcoin['High'], bins=20, color='turquoise')

# Add labels for both axes
plt.xlabel('Daily High')
plt.ylabel('Count')

In [ ]:
# create style
style.use("seaborn-whitegrid")

# set width and height
plt.figure(figsize=(10,6))

# Add a title
plt.title("Frequency Distribution of 'Low' Prices")

# Show the average daily low price
sns.histplot(data=bitcoin, x=bitcoin['Low'], bins=20, color='turquoise')

# Add labels for both axes
plt.xlabel('Daily Low')
plt.ylabel('Count')

In [ ]:
# create style
style.use("seaborn-whitegrid")

# set width and height
plt.figure(figsize=(10,6))

# Add a title
plt.title("Frequency Distribution of Daily Volume")

# Show the average daily volume
#sns.histplot(data=bitcoin, x=bitcoin['Volume'], bins=20, color='limegreen')
sns.kdeplot(data=bitcoin, x=bitcoin['Volume'], color='limegreen', shade=True)

# Add labels for both axes
plt.xlabel('Volume')
plt.ylabel('Count')

The bulk of the daily volume of transactions rarely exceeds 100 billion but there are a few exceptions. 

Given the mean values for OHLC prices, another way to find out how many values are above or below the average would be:

In [ ]:
bitcoin['Open'].loc[bitcoin['Open'] > 29707.22]

In [ ]:
tot_o = (121 / 162) * 100
print(tot_o)

In [ ]:
bitcoin['High'].loc[bitcoin['High'] > 30555.68]

So there are 121 values greater than the mean daily opening price. Calculating this as a percentage of the total number of observations gives:

In [ ]:
tot_h = (123 / 162) * 100
print(tot_h)

Repeating this step for the other price attributes gives:

In [ ]:
bitcoin['Low'].loc[bitcoin['Low'] > 28874.21]

In [ ]:
tot_l = (122 / 162) * 100
print(tot_l)

In [ ]:
bitcoin['Close'].loc[bitcoin['Close'] > 29864.75]

In [ ]:
tot_c = (121 / 162) * 100
print(tot_c)

I can tell for all of these price categories that roughly 75% of the values lie above the mean which is another way to show their frequency distributions are positively skewed.

# Feature Selection
Choosing the right data columns (features) for the Input set:

In [ ]:
# import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# import data
bitcoin = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")
df = pd.DataFrame(bitcoin).dropna(axis=0)

Checking the data in the first couple of rows:

In [ ]:
df.head(2)

In [ ]:
# all column names
df.columns

In [ ]:
# all column data types
df.dtypes

Next I want to split the data into predictors and a target variable, containing all my feature columns in one matrix or Input variable (X) and the Target or Output variable in a column vector (y). Assigning the dependant, or y variable for the modelling process to the daily close price information (which will represent the labelled data), I have:

In [ ]:
y = df['Close']
print(y)

The features are a selection of columns used to predict 'y', also known as the independent variables. I am choosing to leave the 'Date' and 'Adj Close' columns out of the X variable dataframe.

Note, I can either store the individual features in a variable which can be referenced or called when performing some function, or I can store the exact feature names as a list in the dataframe. For example:

In [ ]:
# set the feature columns list
bitcoin_features = ['Open','High','Low','Volume']
print(bitcoin_features)

In [ ]:
# select features as dataframe
X = df[bitcoin_features]
print(X)

In [ ]:
# an alternative way
X = df[["Open", "High", "Low", "Volume"]]
print(X)

A couple of important things to note here. Firstly because I already dropped not-available row entries (3), there are 362 correct entries spanning 365 rows which is correct. I don't need to perform this dropna() method on X and y individually because I have already applied this operation to the dataframe (df).

Secondly, I can see that referencing the features and storing them in a separate variable named 'bitcoin_features' really only comes in handy when there are a large number of features, perhaps too many to type into a list; but for the purpose of this exercise I prefer entering each feature name individually.

## Splitting the Data
Split the data into training and test sets with a 70-30 split but not without making a copy of the dataframe first.

In [ ]:
df = df.copy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Printing out the shape of the training sets for both the X matrix and y vector gives:

In [ ]:
print(X_train.shape)
print(y_train.shape)

And the shape of the test data:

In [ ]:
print(X_test.shape)
print(y_test.shape)

So I can see that 253/362 * 100 = 70% and 109/362 * 100 = 30% for both the train and test sets respectively. Next I will save a copy of the dataframe to use.

# Model Selection (Linear Regression)
This enables me to explore the linear nature of the relationship between the input and output data and fit a regression line using a scatter plot.

## Training the Data and Fitting

In [ ]:
# instantiate model
linear_regression = LinearRegression()

# fit model
linear_regression.fit(X_train, y_train)

Now the 'training' data has been fit, try making a prediction on the first 4 values in the test set first. This includes all the values in row one of X.

In [ ]:
price_predictions = linear_regression.predict(X_test)
print("Predictions: ", linear_regression.predict(X_test.iloc[:4]))

Comparing these values to the actual test set values:

In [ ]:
X_test.head(1)

In [ ]:
# I want to predict the close price based on the values from line 163 in the table above
linear_regression.predict([[14354.31, 14506.5, 13948.08, 3.611139e+10]])

Comparing this forecast value for y to the actual value for y:

In [ ]:
y_actual = 14115.09

# Assuming y = a + b1x1 + b2x2 + b3x3 + b4x4
# where:
# a = intercept (bias term)
# b = coefficient value
# x = feature value

y = 107.8575212006981 + (-4.38810208e-01*14354.31) + (9.50132416e-01*14506.5) + (4.76889709e-01*13948.08) + (-2.42699860e-10*3.611139e+10)

print(y)

So I can see that C$ 14,115.09 is reasonably close to the predicted value for y, but not that great! The Linear Regression prediction value is almost identical to my prediction model for y so I know the formula works.

## Metrics
So checking the values against the BTC_CAD.csv dataset I can see they are not exactly accurate. One way to check is introduce an accuracy score called MSE (mean squared error), but first I will check the R-squared measure to establish the overall degree of fit to the line.

In [ ]:
print("R-squared: ", linear_regression.score(X_test, y_test))

This is a fairly high score and I can see this relationship in a scatter plot showing actual prices against predicted prices.

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(y_test, price_predictions)
print(mse)

rmse = np.sqrt(mse)
print(rmse)

# set the width and height of the plot
plt.figure(figsize=(7,7))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, price_predictions)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=price_predictions)

To find the intercept and coefficients:

In [ ]:
print(linear_regression.intercept_)
print(linear_regression.coef_)

This output provides me with the intercept value which intersects the y-axis at 107.85 and the respective coefficient values for each independent variable from x1 to x4 (open, high, low and volume values).

# Evaluation
My first look at the results of the scores and their accuracy measures prompted me to re-visit the code and try and introduce more efficiency (reducing the number of lines), but also more consistency such as replicating similar steps for the use of different types of model. So even though I started storing the same dataset in different variables it can quickly become confusing and simultaneously very important to ensure some similarity exists with regard to training, testing and evaluation/performance measurement among the different models. 

These procedures will require more modification when I start to apply Polynomial Regression and tackle any problems attributed to over or under-fitting of data with the application of regularization.

Initial linear regression rmse score = 689.1925598643533
R-squared accuracy = 0.9991392014437468

In conclusion, the model requires a much larger dataset and would benefit from a larger time period of price information, perhaps dating back to it's initial inception (2010 - not that long ago). Because the dataset is too small it falls prey to 'Sampling Noise' which is the degree of error from sampling only one subsection of the population. The model could be improved if other sample sizes were used or if the entire population of data for Bitcoin prices was used. 

Regularization implies reducing the number of features or creating a more efficient use of feature engineering to counteract overfitting the data and should be applied to improve this problem. As well as more price data another way to achieve better generalization on unseen data may involve using features from substitute investments or alternative crypto-currencies. It might pay to include market indices relevant to the sector, or even some form of sentiment analysis from market news which could be interpreted numerically using a form of one-hot encoding. 

This model could potentially incorporate the risk from substitute investments (Dogecoin for example) which are direct competitors or alternative investments (such as Gold) which are a vital component of several investment portfolios, the risk associated with the industry or market sector, political risk associated with foreign and domestic policy, or even corporate risk associated with management level decisions such as whether or not Blockchain technology could be under threat of being potentially unravelled by Quantum computing. 

All of these components combined would no doubt provide a better overall picture and contribute to more accurate price discovery in the Crypto markets. Next, I will try to fit a forecast line using a Polynomial Regression model but for now let's print out dependencies which allow this notebook to be reproducable across different platforms so others who are interested can view these findings.

# Print dependencies
Dependencies are fundamental to record the **computational environment**.   

- Use [watermark](https://github.com/rasbt/watermark) to print version of python, ipython, and packages, and characteristics of the computer

In [ ]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,pandas,numpy,watermark,tarfile,urllib3,matplotlib,seaborn,sklearn,pickle5 

# date
print (" ")
%watermark -u -n -t -z